#Download e Preparação do Dataset

In [4]:
!pip install ultralytics split-folders kaggle
# !mkdir /content/drive/MyDrive/Colab\ Notebooks/VisaoCompu/AV3/
# !mv /content/drive/MyDrive/vehicle_damage_split /content/drive/MyDrive/Colab\ Notebooks/VisaoCompu/AV3/


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 130.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 112.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
# Cópia dataset par sessão do colab para acesso mais rápido
!cp -r "/content/drive/MyDrive/Colab Notebooks/VisaoCompu/AV3/vehicle_damage_split" /content/

In [ ]:
import kagglehub
import splitfolders
import os

# 1. Baixar o dataset do Kaggle
# print("Baixando o dataset...")
# path = kagglehub.dataset_download("gauravduttakiit/vehicle-damage-identification")
# print(f"Dataset baixado em: {path}")

# O dataset vem compactado, então precisamos descompactá-lo.
# O kagglehub já pode fazer isso, mas vamos garantir o caminho correto.
# A estrutura é path/train e path/test
# original_train_dir = os.path.join(path, "train")
# output_split_dir = "vehicle_damage_split" # Nova pasta para os dados divididos

# # 2. Dividir a pasta 'train' em 'train' e 'val' (80/20)
# print(f"Dividindo os dados de '{original_train_dir}' em treino e validação...")
# splitfolders.ratio(
#     original_train_dir,
#     output=output_split_dir,
#     seed=42, # Para resultados reproduzíveis
#     ratio=(0.8, 0.2), # 80% para treino, 20% para validação
#     group_prefix=None,
#     move=False # Copia os arquivos em vez de movê-los
# )

# print(f"Dados divididos e salvos em '{output_split_dir}'")
# # Agora você terá uma pasta 'vehicle_damage_split' com as subpastas 'train' e 'val'.

In [ ]:
import os
os.environ['TORCH_ENABLE_DYNAMIC_SHAPES'] = 'False'

# Treinamento do Modelo YOLOv8 de Classificação

In [ ]:
from ultralytics import YOLO

model = YOLO('yolo11n-cls.pt')

# Caminho para o dataset copiado localmente
local_data_path = '/content/vehicle_damage_split'

# Verifique se o diretório de dados existe
if not os.path.exists(local_data_path):
    print(f"Erro: O diretório de dados {local_data_path} não foi encontrado.")
    print("Certifique-se de executar a célula com o comando 'cp' para copiar os dados do Google Drive primeiro.")
else:
    results = model.train(
        # --- Caminho do dataset e projeto ---
        data=local_data_path,
        project='/content/drive/MyDrive/Colab Notebooks/VisaoCompu/AV3/training_reuslts',
        name='vehicle_damage_cls_fast',

        # --- Parâmetros de performance ---
        imgsz=224,
        epochs=100,
        batch=512,          
        workers=16,          # Aumenta a velocidade de carregamento dos dados
        cache=True,         # Coloca o dataset na RAM para acesso ultra-rápido após a 1ª época//não é usado pois tem vazamento de memória
        amp=True,           # Ativa o Treinamento com Precisão Mista

        # --- Outros parâmetros ---
        optimizer='auto',   # Deixa a biblioteca escolher o melhor otimizador (geralmente AdamW)
        patience=20
    )

    print("Treinamento concluído!")

Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=512, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/vehicle_damage_split, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=vehicle_damage_cls_fast2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.0, plots=Tru

train: Scanning /content/vehicle_damage_split/train... 4605 images, 0 corrupt: 100%|██████████| 4605/4605 [00:00<?, ?it/s]


WARNING ⚠️ Classification `cache_ram` training has known memory leak in https://github.com/ultralytics/ultralytics/issues/9824, setting `cache_ram=False`.
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 778.9±670.2 MB/s, size: 105.0 KB)


val: Scanning /content/vehicle_damage_split/val... 1155 images, 0 corrupt: 100%|██████████| 1155/1155 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.004), 40 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 2 dataloader workers
Logging results to /content/drive/MyDrive/Colab Notebooks/VisaoCompu/AV3/training_reuslts/vehicle_damage_cls_fast2
Starting training for 100 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...


      Epoch    GPU_mem       loss  Instances       Size


     62/100      6.03G    0.02611        509        224: 100%|██████████| 9/9 [01:09<00:00,  7.75s/it]
               classes   top1_acc   top5_acc: 100%|██████████| 2/2 [00:00<00:00,  6.23it/s]

                   all      0.965      0.999



      Epoch    GPU_mem       loss  Instances       Size


     63/100      6.03G    0.02535        512        224:  67%|██████▋   | 6/9 [00:55<00:20,  6.95s/it]

#Avaliação e Predição

In [3]:
!wget -o teste_img.jpeg "https://tatudoseguro.com/wp-content/uploads/2025/01/963550d4-02c0-44d2-9722-1569b11c127b.jpeg"

In [5]:
from ultralytics import YOLO
import os

# 1. Carregar o modelo treinado
trained_model = YOLO('/content/drive/MyDrive/Colab Notebooks/VisaoCompu/AV3/training_reuslts/vehicle_damage_cls_fast2/weights/best.pt')

# 2. Fazer uma predição em uma imagem de teste
# Vamos pegar uma imagem de exemplo
test_image_path = "/content/teste.jpeg"

if os.path.exists(test_image_path):
    # Fazer a predição
    results = trained_model(test_image_path)

    # 3. Exibir os resultados
    # 'names' é um dicionário mapeando o índice da classe para o nome da classe
    class_names = results[0].names

    # 'probs' contém as probabilidades para cada classe
    top1_index = results[0].probs.top1
    top1_confidence = results[0].probs.top1conf

    predicted_class_name = class_names[top1_index]

    print(f"Imagem: {test_image_path}")
    print(f"Classe Prevista: {predicted_class_name}")
    print(f"Confiança: {top1_confidence.item():.2f}")
else:
    print(f"Caminho da imagem de teste não encontrado: {test_image_path}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

image 1/1 /content/teste.jpeg: 224x224 glass shatter 1.00, crack 0.00, dent 0.00, tire flat 0.00, lamp broken 0.00, 4.7ms
Speed: 23.1ms preprocess, 4.7ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)
Imagem: /content/teste.jpeg
Classe Prevista: glass shatter
Confiança: 1.00


In [ ]:
results = trained_model.val(
    data='/content/drive/MyDrive/Colab Notebooks/VisaoCompu/AV3/vehicle_damage_split', # Especifica o conjunto de validação
    imgsz=224,
    project='/content/drive/MyDrive/Colab Notebooks/VisaoCompu/AV3/validation_results', # Salva os resultados em uma nova pasta
    name='vehicle_damage_val'
)

Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
train: /content/drive/MyDrive/Colab Notebooks/VisaoCompu/AV3/vehicle_damage_split/train... found 4605 images in 6 classes ✅ 
val: /content/drive/MyDrive/Colab Notebooks/VisaoCompu/AV3/vehicle_damage_split/val... found 1155 images in 6 classes ✅ 
test: None...
val: Fast image access ✅ (ping: 0.8±0.2 ms, read: 0.3±0.2 MB/s, size: 105.0 KB)


val: Scanning /content/drive/MyDrive/Colab Notebooks/VisaoCompu/AV3/vehicle_damage_split/val... 1155 images, 0 corrupt: 100%|██████████| 1155/1155 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc:   1%|▏         | 1/73 [00:03<04:38,  3.87s/it]


100%|██████████| 755k/755k [00:00<00:00, 15.8MB/s]
               classes   top1_acc   top5_acc: 100%|██████████| 73/73 [02:31<00:00,  2.07s/it]


                   all      0.975          1
Speed: 0.0ms preprocess, 22.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/drive/MyDrive/Colab Notebooks/VisaoCompu/AV3/validation_results/vehicle_damage_val2
